In [1]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
import os

In [2]:
# Load data from data folder
def get_data(folder_path):
    os.chdir(folder_path)
    df = pd.DataFrame()

    for file in os.listdir(folder_path):
        file_path = f"{folder_path}/{file}"
        print(f'reading {file_path}')
        data=pd.read_csv(file_path, encoding="ISO-8859-1")
        print(data.shape)
        df = pd.concat([df, data])

    return df

current_path = os.getcwd()
folder_path = current_path + "/data"
df = get_data(folder_path)
df.shape

c:\Users\Michelle\Documents\SeeTrue\nlp
reading c:\Users\Michelle\Documents\SeeTrue\nlp/data/ClaimFakeCOVID-19_5.csv
(27, 2)
reading c:\Users\Michelle\Documents\SeeTrue\nlp/data/ClaimFakeCOVID-19_7.csv
(1, 2)
reading c:\Users\Michelle\Documents\SeeTrue\nlp/data/ClaimRealCOVID-19.csv
(116, 2)
reading c:\Users\Michelle\Documents\SeeTrue\nlp/data/ClaimRealCOVID-19_5.csv
(166, 2)
reading c:\Users\Michelle\Documents\SeeTrue\nlp/data/ClaimRealCOVID-19_7.csv
(172, 2)
reading c:\Users\Michelle\Documents\SeeTrue\nlp/data/lynette.csv
(1845, 2)
reading c:\Users\Michelle\Documents\SeeTrue\nlp/data/NewsFakeCOVID-19.csv
(56, 2)
reading c:\Users\Michelle\Documents\SeeTrue\nlp/data/NewsFakeCOVID-19_5.csv
(572, 2)
reading c:\Users\Michelle\Documents\SeeTrue\nlp/data/NewsFakeCOVID-19_7.csv
(266, 2)
reading c:\Users\Michelle\Documents\SeeTrue\nlp/data/NewsRealCOVID-19.csv
(893, 2)
reading c:\Users\Michelle\Documents\SeeTrue\nlp/data/NewsRealCOVID-19_5.csv
(1590, 2)
reading c:\Users\Michelle\Documents\See

(13160, 2)

In [3]:
# Visualise data
df.groupby(['category']).size().plot.bar()

: 

: 

In [3]:
# use bert-base-multilingual-cased model, if dataset is not in English
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# maps category in dataframe into id representation
labels = {'FAKE':0,
          'REAL':1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [4]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2) # no of categories
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [6]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  

In [7]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')

In [10]:
# Split data into training, validation, and test set with the proportion of 80:10:10

np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

1486 186 186


In [11]:
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  7%|▋         | 54/743 [08:10<1:44:21,  9.09s/it]


KeyboardInterrupt: 

In [11]:
evaluate(model, df_test)

Test Accuracy:  0.996
